# Gate Model

The `Gate` class represents a controllable gate in a water management system. It models the outflow based on the upstream and downstream water levels, and the gate's opening height. This notebook explains how to use the `Gate` model and simulates its behavior under changing control signals.

## State Variables

The state of the gate is described by:

- `opening` (float): The current opening height of the gate (m).
- `outflow` (float): The outflow rate through the gate (m^3/s).

## Parameters

The gate's physical and operational characteristics are defined by:

- `discharge_coefficient` (float): A dimensionless coefficient representing the efficiency of the gate's flow.
- `width` (float): The width of the gate (m).
- `max_opening` (float): The maximum possible opening height of the gate (m).
- `max_rate_of_change` (float): The maximum speed at which the gate opening can change (m/s).

## Simulation Example

The following example demonstrates how to create a `Gate` instance and simulate its operation. The simulation shows the gate opening to a target level, and then closing. The outflow is calculated at each step based on assumed constant water levels.

**Note:** To run this notebook, ensure you have `matplotlib` installed (`pip install matplotlib`) and that you are running the Jupyter server from the root directory of the project.

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.gate import Gate
from swp.core.interfaces import State, Parameters

# Initial state and parameters for the gate
initial_state = State(opening=0.0, outflow=0.0)
parameters = Parameters(
    discharge_coefficient=0.61,
    width=5.0,              # meters
    max_opening=2.0,        # meters
    max_rate_of_change=0.05 # m/s
)

# Create a Gate instance
gate = Gate(name="control_gate", initial_state=initial_state, parameters=parameters)

# Simulation settings
dt = 1  # time step in seconds
simulation_duration = 120  # 2 minutes
num_steps = int(simulation_duration / dt)

# Store results for plotting
history = []

# Define external conditions and control signals for the simulation
upstream_water_level = 10.0  # constant upstream water level (m)
downstream_water_level = 8.0   # constant downstream water level (m)

# Run the simulation loop
for t in range(num_steps):
    # Define the control signal based on time
    if t < 20:
        target_opening = 1.5  # Command to open
    elif t > 80:
        target_opening = 0.5  # Command to partially close
    else:
        target_opening = 1.5 # Maintain opening
        
    action = {
        'control_signal': target_opening,
        'upstream_head': upstream_water_level,
        'downstream_head': downstream_water_level
    }
    
    current_state = gate.step(action=action, dt=dt)
    history.append(current_state.copy())

# Print the final state
print("Final State:", gate.get_state())

# Prepare data for plotting
time_steps = [i * dt for i in range(num_steps)]
openings = [s['opening'] for s in history]
outflows = [s['outflow'] for s in history]

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

ax1.plot(time_steps, openings, label='Gate Opening')
ax1.set_ylabel('Opening (m)')
ax1.set_title('Gate Operation Simulation')
ax1.axhline(1.5, color='r', linestyle='--', label='Target Opening (1.5m)')
ax1.axhline(0.5, color='g', linestyle='--', label='Target Opening (0.5m)')
ax1.grid(True)
ax1.legend()

ax2.plot(time_steps, outflows, label='Outflow', color='orange')
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Outflow (m^3/s)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
